## Team Operation: Reset, Stop, Resume and Abort
- to take more control like a teamlead

In [4]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from dotenv import load_dotenv
load_dotenv()
import os

key = os.getenv("Groq_api_key")

model_client =  OpenAIChatCompletionClient(
    base_url="https://api.groq.com/openai/v1",
    model="llama-3.3-70b-versatile",
    api_key = key,
    model_info={
        "family":'llama',
        "vision" :False,
        "function_calling":True,
        "json_output": True
    }
)


# 3 AGENTS
add_1_agent_first = AssistantAgent(
    name="add_1_agent_first",
    model_client=model_client,
    system_message="add 1 to number which is given to you as input and give out resultant number. start with 0 if no input is given"
)

add_1_agent_second = AssistantAgent(
    name="add_1_agent_second",
    model_client=model_client,
    system_message="add 1 to number which you got from previous agent and give resultant number."
)

add_1_agent_third = AssistantAgent(
    name="add_1_agent_third",
    model_client=model_client,
    system_message="add 1 to number which you got from previous agent and give resultant number."
)

c:\Users\shubu\Desktop\AutoGen Hardcode\venv\lib\site-packages\autogen_ext\models\openai\_openai_client.py:413: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


In [5]:
# TEAM CREATED
from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    [add_1_agent_first, add_1_agent_second, add_1_agent_third], # list of agents 
    max_turns=3
)

In [7]:
from autogen_agentchat.ui import Console

await Console(team.run_stream(task= "first number is 5"))

---------- TextMessage (user) ----------
first number is 5


---------- TextMessage (add_1_agent_first) ----------
The number given is 5. Adding 1 to it makes it 6.

Resultant number: 6
---------- TextMessage (add_1_agent_second) ----------
The number given is 6. Adding 1 to it makes it 7.

Resultant number: 7
---------- TextMessage (add_1_agent_third) ----------
The number given is 7. Adding 1 to it makes it 8.

Resultant number: 8


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='first number is 5', type='TextMessage'), TextMessage(source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=160, completion_tokens=24), metadata={}, content='The number given is 5. Adding 1 to it makes it 6.\n\nResultant number: 6', type='TextMessage'), TextMessage(source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=177, completion_tokens=24), metadata={}, content='The number given is 6. Adding 1 to it makes it 7.\n\nResultant number: 7', type='TextMessage'), TextMessage(source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=205, completion_tokens=24), metadata={}, content='The number given is 7. Adding 1 to it makes it 8.\n\nResultant number: 8', type='TextMessage')], stop_reason='Maximum number of turns 3 reached.')

## Resuming a Team
Teams are stateful and maintains the conversation history and context after each run, unless you reset the team.


We can resume a team to continue from where it left off by calling the run() or run_stream() method without a **new task**


In [ ]:
await Console(team.run_stream()) # first it will continue with the previous state

---------- TextMessage (add_1_agent_first) ----------
The number given is 8. Adding 1 to it makes it 9.

Resultant number: 9
---------- TextMessage (add_1_agent_second) ----------
The number given is 9. Adding 1 to it makes it 10.

Resultant number: 10
---------- TextMessage (add_1_agent_third) ----------
The number given is 10. Adding 1 to it makes it 11.

Resultant number: 11


TaskResult(messages=[TextMessage(source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=244, completion_tokens=24), metadata={}, content='The number given is 8. Adding 1 to it makes it 9.\n\nResultant number: 9', type='TextMessage'), TextMessage(source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=261, completion_tokens=24), metadata={}, content='The number given is 9. Adding 1 to it makes it 10.\n\nResultant number: 10', type='TextMessage'), TextMessage(source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=289, completion_tokens=24), metadata={}, content='The number given is 10. Adding 1 to it makes it 11.\n\nResultant number: 11', type='TextMessage')], stop_reason='Maximum number of turns 3 reached.')

### team resumed from where it left off in the output above, and the first message is from the next agent after the last agent that spoke before the team stopped.

In [ ]:
  # second we can give a new task and it will have idea about previous task
await Console(team.run_stream(task = 'What was the largest number you got in the result?'))

---------- TextMessage (user) ----------
What was the largest number you got in the result?
---------- TextMessage (add_1_agent_first) ----------
Let me recall the sequence:

The sequence started with 0, and then it went as follows: 
0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11.

The largest number I got in the result was 11.
---------- TextMessage (add_1_agent_second) ----------
The correct sequence was indeed: 0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11.

And yes, the largest number you got in the result was indeed 11. 

However, I noticed that the sequence initially started with 0, then after 3, it jumped to 5 instead of 4. It would be more logical if the sequence had continued from 3 to 4.
---------- TextMessage (add_1_agent_third) ----------
You're right, the sequence did jump from 3 to 5, skipping 4. The correct sequence should have been:

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11

If we had continued the sequence correctly, the largest number I would have gotten in the result would still be 11, but the s

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What was the largest number you got in the result?', type='TextMessage'), TextMessage(source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=344, completion_tokens=66), metadata={}, content='Let me recall the sequence:\n\nThe sequence started with 0, and then it went as follows: \n0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11.\n\nThe largest number I got in the result was 11.', type='TextMessage'), TextMessage(source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=403, completion_tokens=102), metadata={}, content='The correct sequence was indeed: 0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11.\n\nAnd yes, the largest number you got in the result was indeed 11. \n\nHowever, I noticed that the sequence initially started with 0, then after 3, it jumped to 5 instead of 4. It would be more logical if the sequence had continued from 3 to 4.', type='TextMessage'), TextMessage(source='add_1_agent_third',

## Reset our Team

In [10]:
await team.reset() # on_reset() on all agents
await Console(team.run_stream())

---------- TextMessage (add_1_agent_first) ----------
Since no input is given, I'll start with 0. 

0 + 1 = 1

The resultant number is: 1
---------- TextMessage (add_1_agent_second) ----------
Since the previous number was 1, I will add 1 to it.

1 + 1 = 2

The resultant number is: 2
---------- TextMessage (add_1_agent_third) ----------
Since the previous number was 2, I will add 1 to it.

2 + 1 = 3

The resultant number is: 3


TaskResult(messages=[TextMessage(source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=57, completion_tokens=30), metadata={}, content="Since no input is given, I'll start with 0. \n\n0 + 1 = 1\n\nThe resultant number is: 1", type='TextMessage'), TextMessage(source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=80, completion_tokens=32), metadata={}, content='Since the previous number was 1, I will add 1 to it.\n\n1 + 1 = 2\n\nThe resultant number is: 2', type='TextMessage'), TextMessage(source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=116, completion_tokens=32), metadata={}, content='Since the previous number was 2, I will add 1 to it.\n\n2 + 1 = 3\n\nThe resultant number is: 3', type='TextMessage')], stop_reason='Maximum number of turns 3 reached.')

### for STOP learn TERMINATION

## Aborting A Team
- Different from stopping a team, aborting a team will immediately stop the team and raise a CancelledError exception.

## **DONE IN TERMINATION**